In [ ]:
import sys
sys.path.append('../')
from xd import *
from utils import *
import esutil
import healpy as hp
from systematics import *
from systematics_module import *
import os

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from run_systematics import sys_iteration, weightmultiply, fitting_allSP, calling_sysMap

In [ ]:
# calling map 
path = '/fs/scratch/PCON0003/warner785/bwarner/'
LSSGoldmask = fitsio.read(path+'MASK_Y3LSSBAOSOF_22_3_v2p2.fits')
# 'Y1LSSmask_v2_redlimcut_il22_seeil4.0_4096ring.fits'
#GoldMask = callingEliGoldMask()
LSSMask = LSSGoldmask
GoldMask = LSSGoldmask

pixra, pixdec = hp.pix2ang(nside=4096,ipix=GoldMask['PIXEL'],nest=True,lonlat=True)

LSSMask = LSSMask[pixdec >-3.0 ]
#GoldMask_st82 = Cuts.SpatialCuts(GoldMask, ra=320, ra2=360, dec=-2, dec2=2)
GoldMask_st82 = GoldMask[ pixdec > -3.0 ]
GoldMask_spt = GoldMask[ pixdec < -3.0 ]
#GoldMask_spt = Cuts.SpatialCuts(GoldMask_spt, ra=0, ra2 = 100, dec=-52, dec2 = -48)

pixarea = hp.nside2pixarea( 4096, degrees = True)
sptnpix = GoldMask_spt['PIXEL'].size #hp.get_map_size( GoldMask_spt['PIXEL'] )
st82npix =  GoldMask_st82['PIXEL'].size # hp.get_map_size( GoldMask_st82 )
SPTMaparea = pixarea * sptnpix
ST82Maparea = pixarea * st82npix

In [ ]:
sys.path.append('../../')
sys.path.append('/DMASS-analysis/measurements/code_py3/')
import esutil
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
os.chdir('../../DMASS-analysis/measurements/code_py3/')
print(os.getcwd())
from calling_catalogs import *
rootdir = '/users/PCON0003/warner785/DMASS-analysis/measurements/clustering/'
from ggl_mcal import run_ng_jk, run_nn_jk, run_nk_jk, compute_eix_jk, compute_Rgamma_jk, construct_jk, compute_jkcov, save_results

In [ ]:
def keepGoodRegion(des, hpInd = False, balrog=None):
    import healpy as hp
    import fitsio
    # 25 is the faintest object detected by DES
    # objects larger than 25 considered as Noise
    
    path = '/fs/scratch/PCON0003/warner785/bwarner/'
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v2_il22_seeil4.0_nside4096ring_redlimcut.fits')
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits')
    LSSGoldmask = fitsio.read(path+'MASK_Y3LSSBAOSOF_22_3_v2p2.fits')
    ringhp = hp.nest2ring(4096, [LSSGoldmask['PIXEL']])
    #Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits
    #frac_cut = LSSGoldmask['FRAC'] > 0.8
    #ind_good_ring = LSSGoldmask['PIXEL'][frac_cut]
    ind_good_ring = ringhp
    
    # healpixify the catalog.
    nside=4096
    # Convert silly ra/dec to silly HP angular coordinates.
    phi = des['RA'] * np.pi / 180.0
    theta = ( 90.0 - des['DEC'] ) * np.pi/180.0

    hpInd = hp.ang2pix(nside,theta,phi,nest=False)
    keep = np.in1d(hpInd, ind_good_ring)
    des = des[keep]
    if hpInd is True:
        return ind_good_ring
    else:
        return des

In [ ]:
def ra_dec_to_xyz(ra, dec):
    """Convert ra & dec to Euclidean points
    Parameters
    ----------
    ra, dec : ndarrays
    Returns
    x, y, z : ndarrays
    """
    sin_ra = np.sin(ra * np.pi / 180.)
    cos_ra = np.cos(ra * np.pi / 180.)

    sin_dec = np.sin(np.pi / 2 - dec * np.pi / 180.)
    cos_dec = np.cos(np.pi / 2 - dec * np.pi / 180.)

    return (cos_ra * sin_dec,
            sin_ra * sin_dec,
            cos_dec)

def uniform_sphere(RAlim, DEClim, size=1):
    """Draw a uniform sample on a sphere
    Parameters
    ----------
    RAlim : tuple
        select Right Ascension between RAlim[0] and RAlim[1]
        units are degrees
    DEClim : tuple
        select Declination between DEClim[0] and DEClim[1]
    size : int (optional)
        the size of the random arrays to return (default = 1)
    Returns
    -------
    RA, DEC : ndarray
        the random sample on the sphere within the given limits.
        arrays have shape equal to size.
    """
    zlim = np.sin(np.pi * np.asarray(DEClim) / 180.)

    z = zlim[0] + (zlim[1] - zlim[0]) * np.random.random(size)
    DEC = (180. / np.pi) * np.arcsin(z)
    RA = RAlim[0] + (RAlim[1] - RAlim[0]) * np.random.random(size)
    
    return RA, DEC

def uniform_random_on_sphere(data, size = None ):
    ra = data['RA']
    dec = data['DEC']
    
    n_features = ra.size
    #size = 100 * data.size
    
    # draw a random sample with N points
    ra_R, dec_R = uniform_sphere((min(ra), max(ra)),
                                 (min(dec), max(dec)),
                                 size)
    #data = np.asarray(ra_dec_to_xyz(ra, dec), order='F').T
    #data_R = np.asarray(ra_dec_to_xyz(ra_R, dec_R), order='F').T
    
    #random redshift distribution
    
    data_R = np.zeros((ra_R.size,), dtype=[('RA', 'float'), ('DEC', 'float')])
    data_R['RA'] = ra_R
    data_R['DEC'] = dec_R
                              
    return data_R

## ---------------------------------------------

In [ ]:
print(dmass_val['HPIX_4096']) 

In [ ]:
plt.rcParams.update({
  "text.usetex": False,
  "font.family": "Helvetica"
})

In [ ]:
# calling systematic maps
#sysMap = calling_sysMap( properties=properties, kind='SPT', nside=4096 )

# location of FWHM maps: /fs/scratch/PCON0003/warner785/bwarner/
# y3a2_g_o.4096_t.32768_FWHM.WMEAN_EQU.fits.gz 
# y3a2_i_o.4096_t.32768_FWHM.WMEAN_EQU.fits.gz        
# y3a2_r_o.4096_t.32768_FWHM.WMEAN_EQU.fits.gz        
# y3a2_z_o.4096_t.32768_FWHM.WMEAN_EQU.fits.gz 

# location of other maps: /fs/scratch/PCON0003/warner785/bwarner/y3/
# depending on different bands:

input_path = '/fs/scratch/PCON0003/warner785/bwarner/y3/band_z/'
#y3/band_z/
input_keyword = 'y3a2_z_o.4096_t.32768_FWHM.WMEAN_EQU.fits.gz'
sysMap = io.SearchAndCallFits(path = input_path, keyword = input_keyword)

In [ ]:
# import DMASS in validation region
#lens, randoms = calling_lens_catalog('/fs/scratch/PCON0003/warner785/bwarner/dmass_y1_public_v1.fits')
dmass_val, randoms = calling_lens_catalog('/users/PCON0003/warner785/DMASSY3/output/test/train_cat/y3/dmass_st82_DET200.fits')

random_val = uniform_random_on_sphere(dmass_val, size = 50 * dmass_val.size)
# applying LSS mask 
random_val = keepGoodRegion(random_val)

In [ ]:
random_val = appendColumn(random_val, value=np.ones(random_val.size), name='WEIGHT')
randomsind_val = np.random.choice( random_val.size, size=int(random_val.size/3.) )
random_val = random_val[randomsind_val]

In [ ]:
def doVisualization_ngal(property = None, nside = 1024, kind = 'SPT', suffix='', inputdir = '.', outdir='./'):

    import matplotlib.pyplot as plt
    import numpy as np
    from systematics import SysMapBadRegionMask, loadSystematicMaps, MatchHPArea, chisquare_dof, ReciprocalWeights
    from systematics_module.corr import angular_correlation
    
    
    filters = ['g', 'r', 'i', 'z']
    #filters = ['g']
    
    
    if property is 'NSTARS_allband' :
        nside = 1024
        filters = ['g']
    elif property is 'GE' :
        nside = 512
        filters = ['g']
    elif property in ['SKYBRITEpca0','SKYBRITEpca1','SKYBRITEpca2','SKYBRITEpca3'] :
        nside = 4096
        filters = ['g']     

    fig, ax = plt.subplots(2, 2, figsize = (15, 10))
    ax = ax.ravel()

    for i, filter in enumerate(filters):
        filename = inputdir+'/systematic_'+property+'_'+filter+'_'+kind+'_'+suffix+'.txt'
        #print filename
        data = np.loadtxt(filename)
        bins, Cdensity, Cerr, Cf_area, Bdensity, Berr, Bf_area = [data[:,j] for j in range(data[0].size)]
        Cf_area = Cf_area * 1./Cf_area.max()/5.
        
        
        #zeromaskC, zeromaskB = ( Cdensity != 0.0 )*( Cerr != 0.0 ), (Bdensity != 0.0 )*( Berr != 0.0 )
        zeromaskC = ( Cdensity != 0.0 ) 
        zeromaskB = ( Bdensity != 0.0 )
        #Cdensity, Cbins, Cerr, Cf_area = Cdensity, bins, Cerr, Cf_area #Cdensity[zeromaskC], bins[zeromaskC], Cerr[zeromaskC]
        #C_jkerr = C_jkerr[zeromaskC]
        #Bdensity, Bbins, Berr = Bdensity, bins, Berr #Bdensity[zeromaskB],bins[zeromaskB],Berr[zeromaskB]
        #B_jkerr = B_jkerr[zeromaskB]

        #fitting
        #Cchi, Cchidof = chisquare_dof( bins[zeromaskC], Cdensity[zeromaskC], Cerr[zeromaskC] )
        #Bchi, Bchidof = chisquare_dof( bins[zeromaskB], Bdensity[zeromaskB], Berr[zeromaskB] )
    
        nCbins = np.sum(zeromaskC)
        nBbins = np.sum(zeromaskB)
        Cchi = np.sum( (Cdensity[zeromaskC] - 1.0 * np.ones(nCbins) )**2/Cerr[zeromaskC]**2 )
        Bchi = np.sum( (Bdensity[zeromaskB] - 1.0 * np.ones(nBbins) )**2/Berr[zeromaskB]**2 )


        ax[i].errorbar(bins[zeromaskC] , Cdensity[zeromaskC], yerr = Cerr[zeromaskC], 
            color = 'blue', fmt = '.-', capsize=3, label='DMASS, chi2/dof={:>2.2f}/{:3.0f}'.format(Cchi, nCbins) )
        ax[i].errorbar(bins[zeromaskB]*1.0 , Bdensity[zeromaskB], yerr = Berr[zeromaskB], 
            color = 'red', fmt= '.-', capsize=3, label='Random, chi2/dof={:>2.2f}/{:3.0f}'.format(Bchi, nBbins) )

        #ax[i].bar(Bbins+(bins[1]-bins[0])*0.1, Bf_area[zeromaskB]+0.7, (bins[1]-bins[0]) ,color = 'red', alpha=0.3 )
        
        ax[i].set_xlabel('{}_{} (mean)'.format(property, filter))
        ax[i].set_ylabel('n_gal/n_tot '+str(nside))
        
        ymin, ymax = 0.7, 1.3
        if kind is 'SPT' : ymin, ymax = 0.7, 1.3
        ax[i].set_ylim(ymin, ymax)

        barwidth = (bins[1]-bins[0])
        #if property is 'GE' : 
        #    barwidth = bins[zeromaskC][1:] - bins[zeromaskC][:-1]
        #    print barwidth.size, bins[zeromaskC].size
        ax[i].bar(bins[zeromaskC], Cf_area[zeromaskC]+ymin,barwidth ,color = 'blue', alpha = 0.1 )

        #ax[i].set_xlim(8.2, 8.55)
        ax[i].axhline(1.0,linestyle='--',color='grey')
        ax[i].legend(loc = 'best')
        
        #if property == 'FWHM' : ax[i].set_ylim(0.6, 1.4)
        #if property == 'AIRMASS': ax[i].set_ylim(0.0, 2.0)
        #if property == 'SKYSIGMA': ax[i].set_xlim(12, 18)
        if property == 'NSTARS_allband': ax[i].set_xlim(0.0, 2.0)
        if property == 'NSTARS': ax[i].set_xlim(0.0, 1000)
        if property == 'GE' : 
            ax[i].set_xscale('log')
            #ax[i].set_xlim(0.004, 0.2)
    fig.suptitle('systematic test ({})'.format(kind))
    os.system('mkdir '+outdir)
    figname = outdir+'systematic_'+property+'_'+kind+'_'+suffix+'.png'
    fig.savefig(figname)
    print "saving fig to ", figname

    return 0

In [ ]:
properties = ['DEPTH', 'EXPTIME', 'AIRMASS', 'SKYBRITE', 'FWHM']

In [ ]:
for p in properties+['GE', 'NSTARS_allband'] : 
    doVisualization_ngal(property = p, nside = 4096, kind = kind, suffix='vetoed', \
                         inputdir=sysoutdir, outdir=figoutdir)